In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import random


In [168]:
# option = {  "option_spec": [,]
#             "position": , 
#             "nominal":,
#             "strike":,
#             "expiry":
# }

def startegy(options, S, nominal_id = False):
    profit = 0
    for element in options:
        option_type, call_put = element["option_spec"]
        if nominal_id:
            position, nominal, strike = element["position"], 1, element["strike"]   
        else:
            position, nominal, strike = element["position"], element["nominal"], element["strike"]


        if option_type == "vanilla":
            profit = position * nominal * max(0, call_put * (S - strike))
        elif option_type == "con":
            profit = position * nominal * (call_put * (S - strike) > 0)
        elif option_type == "aon":
            profit = S * position * nominal * (call_put * (S - strike) > 0)

    return profit


def strategy_value(options, S, sigma, df_d, df_f):
    value = 0
    expiry_date = options[0]["expiry"]
    logS = np.log(S)
    for element in options:
        opt_type, sign = element["option_spec"]
        pos, nominal, strike = element["position"], element["nominal"], element["strike"]

        logK = np.log(element["strike"]) 
        d_1 = (logS - logK + np.log(df_f/df_d) + (sigma**2 / 2) * expiry_date) / (sigma * np.sqrt(expiry_date))
        d_2 = d_1 - sigma * np.sqrt(expiry_date)

        n_1 = stats.norm.cdf(sign * d_1)
        print(n_1)
        n_2 = stats.norm.cdf(sign * d_2)
        print(n_2)

        

        if opt_type == "vanilla": 
            value += pos * nominal * sign * (df_f * S * n_1 - df_d * strike * n_2)
        elif opt_type == "con":   
            value += pos * nominal * sign * (df_d * n_2)
        elif opt_type == "aon":
            value += pos * nominal * sign * (df_f * n_1)

    return value

def unique_strikes(options):
    strikes = [e["strike"] for e in options]
    return np.unique(strikes)

def plot_strategy_value(S_range, options, nominal_id = True):
    values = np.array([startegy(options, S, nominal_id) for S in S_range])
    strikes = unique_strikes(options)

    plt.figure(figsize=(8, 5))
    for strike in strikes:
        plt.axvline(x=strike, color='g', linestyle='--', alpha=0.3)
    plt.plot(S_range, values, linestyle='-', linewidth=2, color='r', label="Strategy Value")
    plt.xlabel("S", fontsize=12, fontweight='bold')
    plt.ylabel("Wypłata ze strategii", fontsize=12, fontweight='bold')
    plt.title("Wypłata ze strategii", fontsize=14, fontweight='bold')
    plt.grid(True, linestyle='--', alpha=0.6)
    


def s_pdf(x, S, df_d, df_f, sigma, T):
    rd, rf = np.log(df_d)/T, np.log(df_f)/T
    exponent = -0.5 * ((np.log(x/S) - (rd - rf - (sigma**2)/2) * T) / (sigma * np.sqrt(T)))**2
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(exponent)

def generate_fx_options(num_options=5, expiry=0.5):
    option_types = ["aon", "con", "vanilla"]
    strikes = np.linspace(4.5, 4.75, num_options)
    options = []
    
    for i in range(num_options):
        option = {
            "option_spec": [random.choice(option_types), random.choice([1, -1])],
            "position": random.choice([-1, 1]),
            "nominal": random.uniform(10000, 50000),
            "strike": strikes[i],
            "expiry": expiry
        }
        options.append(option)
    
    return options
